# Chapter24: 分類法
- one-class learning: トレーニングデータは1つのクラスから抽出した標本の集合。そのクラスに属するか否かで分類。異常検知器とか。
- two-class learning: binary classificationとも呼ばれる。陰性と陽性を分類する境界を見つけることが目的。
- multi-class learning: 3つ以上のクラスを互いに分類するためにの境界を見つけること。
## 24.1: 分類の評価


In [1]:
# code 24.1 分類器評価のための関数
def accuracy(truePos, falsePos, trueNeg, falseNeg):
    numerator = truePos + trueNeg # 真陽性 ＋ 真陰性
    denominator = truePos + trueNeg + falsePos + falseNeg # 真陽性 + 真陰性 + 偽陽性 + 偽陰性
    return numerator/denominator # 真陽性 ＋ 真陰性 /(全データ)

def sensitivity(truePos, falseNeg):
    try:
        return truePos/(truePos + falseNeg)
    except ZeroDivisionError:
        return float('nan')

def specificity(trueNeg, falsePos):
    try:
        return trueNeg/(trueNeg+falsePos)
    except ZeroDivisionError:
        return float('nan')

def posPredVal(truePos, falsePos):
    try:
        return truePos(truePos + falsePos)
    except ZeroDivisionError:
        return float('nan')

def negPredVal(trueNeg, falseNeg):
    try:
        return trueNeg/(trueNeg + falseNeg)
    except ZeroDivisionError:
        return float('nan')

def getStats(truePos, falsePos, trueNeg, falseNeg, toPrint=True):
    accur = accuracy(truePos, falsePos, trueNeg, falseNeg)
    sens = sensitivity(truePos, falseNeg)
    spec = specificity(trueNeg, falsePos)
    ppv = posPredVal(truePos, falsePos)
    if toPrint:
        print('Accuracy =', round(accur,3))
        print('sensitivity =',round(sens,3))
        print('specificity =', round(spec,3))
        print('pos. Pred. Val. =',round(ppv,3))
    return (accur,sens,spec,ppv)

## 24.2 ランナーの性別予想

In [2]:
# 17.2
def getBMData(filename):
    """与えられたファイルの内容を読み込む、
       ファイルは次の6要素がカンマで区切られた形式
       0. 氏名(string), 1. 性別(string), 2. 年齢(int), 3. 区分(走/車いす),4. 出身国(string),5. 総合タイム(float)
       返り値: それぞれが6変数のリストのdict(辞書)"""
    data = {}
    f = open(filename)
    line = f.readline()
    data['name'],data['gender'],data['age'] = [],[],[] # 辞書要素をリストで初期化
    data['division'],data['country'],data['time'] = [],[],[]
    while line != '': # 最終行まで読み込む
        split = line.split(',') # カンマで区切りリストで返す
        data['name'].append(split[0])
        data['gender'].append(split[1])
        data['age'].append(split[2])
        data['division'].append(split[3])
        data['country'].append(split[4])
        data['time'].append(float(split[5][:-1])) # 改行\nを取り除く
        line = f.readline()
    f.close()
    return data

In [3]:
# 24.2 標本作成とトレーニングデータ、テストデータへの分割
class Runner(object):
    def __init__(self, gender, age, time):
        self.featureVec = (age,time)
        self.label = gender
    
    def featureDist(self, other):
        dist = 0.0
        for i in range(len(self.featureVec)):
            dist += abs(self.featureVec[i] - other.featureVec[i])**2
        return dist**0.5
    
    def getTime(self):
        return self.featureVec[1]
    
    def getAge(self):
        return self.featureVec[0]
    
    def getLabel(self):
        return self.label
    
    def getFeatures(self):
        return self.featureVec
    
    def __str__(self):
        return str(self.getAge()) + ', ' + str(self.getTime()) + ', ' + self.label
    
def buildMarathonExamples(fileName):
    data = getBMData(fileName)
    examples = []
    for i in range(len(data['age'])):
        a = runner(data['gender'][i], data['age'][i], data['time'][i])
        examples.append(a)
    return examples

import random
def divide80_20(examples):
    sampleIndices = random.sample(range(len(examples)), len(examples)//5) # (80, 20)
    trainingSet, testSet = [],[]
    for i in range(len(examples)):
        if i in sampleIndices:
            testSet.append(examples[i])
        else:
            trainingSet.append(examples[i])
    return trainingSet, testSet

- トレーニングデータ全体の58%のランナーが男性。
- すべてのランナーを男性と予想すれば58％のaccuracyを期待できる⇒基準線として意識する

In [ ]:
def findKNearest(example, exampleSet, k):
    kNearest, distances = [],[]
    # 最初のk個の標本からなるリストと、それらのexampleの距離のリストを作る